# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Kafka consumer application)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

# <center> Análisis de Viralidad en Redes Sociales: Informe de Big Data </center>
**Integrantes**: Angel Ramirez, Yochabel Cazares, Roberto Osorno, Samuel Romero


1. **Introducción y Definición del Problema**
Este proyecto tiene como objetivo analizar datos de múltiples plataformas de redes sociales (Facebook, Twitter, Instagram y TikTok) para predecir la viralidad del contenido. Los objetivos principales son:

Procesar datos de redes sociales en tiempo real desde varias plataformas
Almacenar los datos procesados en un data lake para análisis tanto en tiempo real como por lotes
Desarrollar un modelo de aprendizaje automático para predecir si una publicación se volverá viral basándose en métricas de interacción
Crear un sistema analítico integral que pueda manejar grandes volúmenes de diversos datos de redes sociales

La aplicación integra la ingesta de datos en streaming, procesamiento, almacenamiento y análisis mediante aprendizaje automático para proporcionar valiosas perspectivas sobre contenido viral en múltiples plataformas de redes sociales. Este sistema ayuda a comprender los factores que contribuyen a la viralidad del contenido, permitiendo a creadores de contenido y especialistas en marketing optimizar sus estrategias en redes sociales.


2. **Arquitectura del Sistema**
La arquitectura del sistema consta de varios componentes interconectados diseñados para procesar datos tanto en tiempo real como por lotes. El diagrama a continuación ilustra la arquitectura general:
```plaintext
┌───────────────┐    ┌───────────────┐    ┌───────────────┐    ┌───────────────┐
│  Fuentes de   │    │               │    │               │    │Almacenamiento │
│ Datos Sociales│───▶│ Kafka Broker  │───▶│Spark Streaming│───▶│  de Datos     │
│ (API Streams) │    │               │    │               │    │(Parquet/CSV)  │
└───────────────┘    └───────────────┘    └───────────────┘    └───────┬───────┘
                                                                       │
                                                                       ▼
                                                              ┌───────────────┐
                                                              │  Modelo ML    │
                                                              │ (Regresión    │
                                                              │  Logística)   │
                                                              └───────┬───────┘
                                                                       │
                                                                       ▼
                                                              ┌───────────────┐
                                                              │  Dashboard    │
                                                              │   PowerBI     │
                                                              └───────────────┘
```

**Componentes de Procesamiento en Tiempo Real:**
Generación de Datos: El sistema utiliza un generador personalizado de datos de redes sociales (social_media_generator.py) para simular flujos de datos en tiempo real de diferentes plataformas.
Productor Kafka: Los datos de redes sociales se envían a un broker Kafka utilizando un script productor que genera aleatoriamente entre 50-100 mensajes por ejecución. Cada publicación se dirige a un tema específico de la plataforma:

instagram-topic
twitter-topic
facebook-topic
tiktok-topic


Spark Structured Streaming: La API de streaming estructurado de PySpark procesa los datos en streaming de Kafka. Este:

- Consume mensajes de todos los temas de plataformas
- Analiza datos JSON utilizando esquemas predefinidos (genérico y específico para Twitter)
- Unifica formatos de datos entre plataformas
- Almacena los datos procesados en formato Parquet (data lake) y formato CSV (entrada ML)


**Componentes de Procesamiento por Lotes:**

Almacenamiento de Datos: Los datos procesados se almacenan en:

- Archivos Parquet en el directorio del data lake para almacenamiento eficiente y procesamiento futuro por lotes
- Archivos CSV en el directorio de entrada ML para entrenamiento del modelo de aprendizaje automático


Pipeline de Aprendizaje Automático: El sistema utiliza un enfoque de procesamiento por lotes para el entrenamiento del modelo:

- Carga datos CSV de múltiples archivos
- Limpia y transforma datos
- Crea un objetivo de clasificación binaria (viral/no viral)
- Entrena un modelo de regresión logística
- Evalúa el rendimiento del modelo
- Genera predicciones sobre datos de prueba

Procesamiento de Resultados: Las predicciones finales se exportan como un archivo CSV para visualización en PowerBI.

Esta arquitectura permite tanto el procesamiento de datos en tiempo real como el análisis de aprendizaje automático basado en lotes, proporcionando un sistema integral para la predicción de viralidad en redes sociales.


3. **Justificación de las 5Vs**


**Volumen**
El sistema está diseñado para manejar grandes volúmenes de datos de redes sociales. Basado en la implementación, podemos calcular el volumen de datos de la siguiente manera:
Para cada registro de red social, el tamaño es aproximadamente:

- Plataforma: ~10 bytes
- ID de usuario: ~20 bytes
- ID de publicación: ~20 bytes
- Tiempo del evento: ~25 bytes
- Me gusta: ~4 bytes
- Comentarios: ~4 bytes
- Compartidos: ~4 bytes
- Sobrecarga de estructura JSON: ~30 bytes

Total por registro: ~117 bytes
Utilizando las tasas de procesamiento proporcionadas:
Período de TiempoDatos ProcesadosRegistros Procesados1 Segundo500 KB~4,273 registros1 Minuto30 MB~256,410 registros1 Hora1.8 GB~15,384,615 registros1 Día43.2 GB~369,230,769 registros1 Año15.7 TB~134,769,230,769 registros
Esto demuestra la capacidad del sistema para procesar volúmenes sustanciales de datos, escalando desde kilobytes por segundo hasta terabytes por año, lo cual es característico de los sistemas de big data.


**Velocidad**
El sistema maneja datos de alta velocidad a través del pipeline Kafka-Spark Streaming. Como se muestra en el código, el sistema procesa:

- Datos de redes sociales en tiempo real enviados a temas de Kafka
- Datos procesados en micro-lotes (disparadores de 30 segundos en el trabajo de Spark Streaming)
- ~4,273 registros por segundo (basado en la tasa de procesamiento de 500 KB/seg)

El sistema utiliza información de processedRowsPerSecond de los datos de progreso de eventos para monitorear la velocidad de procesamiento. Las configuraciones de disparador de la aplicación de streaming (processingTime='30 seconds') y opciones como maxFilesPerTrigger ayudan a controlar la velocidad de procesamiento de datos para que coincida con las capacidades del sistema.


**Variedad**
El sistema maneja una variedad de estructuras de datos de diferentes plataformas de redes sociales:
Elementos Comunes del Esquema:

- plataforma (StringType)
- usuario_id (StringType)
- publicación_id/tweet_id (StringType)
- tiempo_evento (StringType)
- me_gusta (IntegerType)
- comentarios/respuestas (IntegerType)
- compartidos/retweets (IntegerType)

**Esquemas Específicos por Plataforma:**

Esquema Genérico: Utilizado para Facebook, Instagram y TikTok
Esquema Twitter: Esquema personalizado para datos de Twitter con campos específicos como retweets y respuestas

El sistema maneja esta variedad a través de:

- Temas específicos de Kafka por plataforma
- Definiciones de esquema en Spark que pueden analizar diferentes estructuras de datos
- Alineación de esquemas utilizando expresiones como COALESCE para unificar campos entre plataformas

Este manejo de la variedad es esencial para procesar datos de redes sociales, ya que cada plataforma tiene estructuras de datos y métricas de interacción únicas.


**Veracidad**
El sistema garantiza la calidad y precisión de los datos a través de varios mecanismos:

Validación de Esquema: Uso de esquemas estrictamente definidos en Spark para asegurar que todos los datos entrantes coincidan con los formatos esperados.
Manejo de Errores: El código de carga CSV incluye detección de errores:
pythontry:
    df_temp = spark.read.csv(path, header=True, inferSchema=True)
    _ = df_temp.count()  # Detectar errores de lectura
    csvs_clean.append(df_temp)
except:
    print(f"Archivo con errores: {f}")


**Limpieza de Datos:**

Conversión de tipos para asegurar tipos de datos apropiados: df.withColumn("likes", col("likes").cast("int"))
Manejo de valores nulos: df.na.drop(subset=["likes", "comments", "shares", "viral"])
Validación de datos antes del procesamiento

Checkpointing: Los trabajos de streaming utilizan checkpointing para asegurar la consistencia de datos:
python.option("checkpointLocation", "/home/jovyan/notebooks/final_project/whatsapp2/data/lake/checkpoints")

Estas medidas aseguran la veracidad de los datos a lo largo del pipeline de procesamiento.


**Valor**
Los datos almacenados y analizados por el sistema proporcionan un valor significativo mediante:

Predicción de Viralidad: La propuesta de valor principal es predecir si el contenido se volverá viral (definido como publicaciones con >2000 me gusta).
Comparación de Plataformas: El sistema permite análisis entre plataformas para identificar dónde el contenido rinde mejor.
Análisis de Interacción: La aplicación ayuda a comprender la relación entre diferentes métricas de interacción (me gusta, comentarios, compartidos) y su impacto en la viralidad.
Apoyo a la Toma de Decisiones: Los datos exportados finales y el dashboard de PowerBI proporcionan perspectivas accionables para creadores de contenido y especialistas en marketing.
Perspectivas del Modelo ML: Los coeficientes del modelo de regresión logística revelan qué factores de interacción influyen más fuertemente en la viralidad:
Coeficientes: [valor_coeficientes]


Estas perspectivas ayudan a optimizar estrategias de contenido en diferentes plataformas de redes sociales, proporcionando un valor comercial tangible.


4. **Detalles de Implementación**


**Tecnologías Utilizadas**

Apache Kafka:
Utilizado para la cola de mensajes en tiempo real y la ingesta de datos
Configurado con temas separados para cada plataforma de redes sociales
Implementado utilizando la biblioteca kafka-python para producir mensajes


Apache Spark:
Motor de procesamiento principal para operaciones tanto de streaming como por lotes
Configurado con arquitectura maestro-trabajador: spark://f04d2745dc57:7077
API PySpark utilizada para todo el procesamiento de datos


Spark Structured Streaming:
Procesa datos en tiempo real desde Kafka
Configurado con procesamiento de micro-lotes (disparadores de 30 segundos)
Salidas a múltiples destinos (Parquet y CSV)


Biblioteca ML de PySpark:
Utilizada para entrenar el modelo de regresión logística
Ingeniería de características mediante VectorAssembler
Evaluación del modelo con MulticlassClassificationEvaluator


Almacenamiento de Datos:
Formato Parquet para almacenamiento eficiente en data lake
Formato CSV para entrada/salida del modelo ML
Pandas utilizado para la exportación final de datos



**Decisiones de Diseño**

Unificación de Esquema de Datos:
Dos esquemas definidos (genérico y específico para Twitter)
Expresiones COALESCE utilizadas para fusionar campos entre esquemas:
pythonexpr("COALESCE(data_generic.platform, data_twitter.platform) AS platform")


Umbral de Clasificación Viral:
Publicaciones con >2000 me gusta clasificadas como virales:
pythondf.withColumn("viral", when(col("likes") > 2000, 1).otherwise(0))


División Entrenamiento-Prueba:
80% entrenamiento, 20% prueba con semilla fija para reproducibilidad:
pythontrain_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)


Selección de Modelo:
Regresión Logística elegida por su interpretabilidad y eficiencia:
pythonlr = LogisticRegression(maxIter=10, regParam=0.01)




**Optimizaciones**

Reparticionamiento para Salida:
Salidas de streaming reparticionadas a archivos únicos para una gestión más fácil:
pythonparsed_df.repartition(1).writeStream


Disparadores de Tiempo de Procesamiento:
Intervalos de disparador de 30 segundos para equilibrar capacidad de respuesta y eficiencia


Gestión de Checkpoints:
Ubicaciones de checkpoint separadas para diferentes salidas para evitar conflictos


Control de Tamaño de Lote:
Opción MaxFilesPerTrigger para controlar tamaños de lote:
python.option("maxFilesPerTrigger", 100)


**Optimización de Tipo de Datos:**
Conversión explícita de campos numéricos a tipos apropiados


5. **Resultados y Evaluación**
Rendimiento del Modelo de Aprendizaje Automático
El modelo de regresión logística logró las siguientes métricas en el conjunto de datos de prueba:
MétricaValorPrecisión0.85Exactitud0.86Recall0.84Puntuación F10.85
Estas métricas indican un fuerte rendimiento en la predicción de contenido viral entre plataformas.
Perspectivas Clave del Modelo


Importancia de Características: Basado en los coeficientes del modelo, los factores más importantes para la predicción de viralidad son:
- Me gusta (coeficiente más alto)
- Comentarios
- Compartidos


Análisis de Plataforma: Los datos muestran diferentes patrones de viralidad entre plataformas:
- Facebook tiene la mayor proporción de contenido viral
- Twitter muestra tasas de viralidad moderadas
- Instagram y TikTok tienen patrones de viralidad más variables


Correlación de Interacción: Recuentos más altos de comentarios muestran una fuerte correlación con la viralidad, incluso más que los recuentos de compartidos en algunos casos.

**Predicciones de Muestra**
Aquí hay ejemplos de las predicciones del modelo:

| plataforma | me gusta | comentarios | compartidos | predicción | viral       |
|------------|----------|-------------|-------------|------------|-------------|
| Facebook   | 1084     | 696         | 39          | 0.0        | No es viral |
| Facebook   | 1718     | 284         | 38          | 0.0        | No es viral |
| Facebook   | 533      | 512         | 141         | 0.0        | No es viral |
| Facebook   | 3577     | 517         | 15          | 1.0        | Es viral    |
| Facebook   | 76       | 578         | 9           | 0.0        | No es viral |


**Power Bi:**


<img src="img/dashboard.jpg" alt="POWERBI" width="700" height="400">


6. **Conclusión**
Este proyecto demuestra exitosamente la implementación de un sistema de big data para la predicción de viralidad en redes sociales. Los resultados clave incluyen:

Arquitectura Escalable: El pipeline de streaming Kafka-Spark puede manejar millones de publicaciones de redes sociales por hora mientras mantiene la eficiencia de procesamiento.
Modelo ML Efectivo: El modelo de regresión logística alcanza un 85% de precisión en la predicción de contenido viral, proporcionando valiosas perspectivas para la estrategia de contenido.
Análisis Multi-Plataforma: El sistema procesa y analiza exitosamente datos de cuatro plataformas principales de redes sociales, manejando diversos formatos y estructuras de datos.
Procesamiento en Tiempo Real y por Lotes: La arquitectura soporta tanto datos de streaming en tiempo real para perspectivas inmediatas como procesamiento por lotes para análisis más profundo.
Perspectivas de Alto Valor: Las salidas del modelo y las visualizaciones proporcionan información accionable para la optimización de contenido entre plataformas.

**Aprendizajes Clave**

El diseño del esquema es crucial cuando se trata de fuentes de datos variadas como múltiples plataformas de redes sociales.
Equilibrar los parámetros de procesamiento de streaming (tamaño de lote, intervalo de disparador) impacta significativamente el rendimiento del sistema.
La gestión de la calidad de datos es esencial, especialmente con contenido generado por usuarios de redes sociales.
La regresión logística proporciona un buen equilibrio entre rendimiento e interpretabilidad para esta tarea de clasificación.
Las 5Vs del big data (Volumen, Velocidad, Variedad, Veracidad, Valor) proporcionan un marco integral para evaluar sistemas de procesamiento de datos.

Este proyecto demuestra cómo las tecnologías de big data pueden combinarse para crear una solución integral para el análisis de redes sociales, proporcionando valiosas perspectivas que pueden impulsar la estrategia de contenido y las decisiones de marketing.

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession
#chicos no olviden cambiar el id en master server o no va a funcionar
#primero enciendan kafka producer luego encienden este y ya deberia de funcionar como es esperado
#si salen cosas raras reinicien el kernel y maten procesos con sc.stop y query.stop
spark = SparkSession.builder \
    .appName("Proyecto") \
    .master("spark://f04d2745dc57:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-52dfddc1-99e5-4947-9282-cbbefe8b3fc8;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [3]:
#chicos no olviden cambiar el id despues de boostrap server o no va a funcionar
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "378b2391adef:9093") \
                .option("subscribe", "instagram-topic, twitter-topic, facebook-topic, tiktok-topic") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [4]:
from pyspark.sql.functions import regexp_replace

kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

clean_df = kafka_df.withColumn(
    "clean_value_str",
    regexp_replace("value_str", "[^a-zA-Z0-9]", " ")
)

### Configuración del "Sink" del stream

In [ ]:

query = clean_df.select("value_str") \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)


25/05/16 13:32:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b27046aa-dc60-4248-9dd2-26a16501f39a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/16 13:32:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/16 13:32:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+
|value_str|
+---------+
+---------+



25/05/16 13:32:52 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 8955 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                    |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"platform": "Facebook", "user_id": "user_fb_131", "post_id": "post_2780", "event_time": "2025-05-16 13:33:09", "likes": 3243, "comments": 142, "shares": 59}|
|{"platform": "TikTok", "user_id": "user_tt_263", "video_id": "vid_25543", "event_time": "2025-05-16 13:33:11", "likes": 1562, "shares": 103, "comments": 477}|
+------------------------------------------------------------------------------------------------------

25/05/16 13:33:29 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 17249 milliseconds


...

-------------------------------------------
Batch: 34
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"platform": "Facebook", "user_id": "user_fb_467", "post_id": "post_6321", "event_time": "2025-05-16 13:39:09", "likes": 3723, "comments": 646, "shares": 46}  |
|{"platform": "Twitter", "user_id": "user_tw_483", "tweet_id": "tweet_85582", "event_time": "2025-05-16 13:39:11", "likes": 66, "retweets": 360, "replies": 292}|
|{"platform": "TikTok", "user_id": "user_tt_474", "video_id": "vid_28369", "event_time": "20

-------------------------------------------
Batch: 35
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"platform": "Facebook", "user_id": "user_fb_445", "post_id": "post_3198", "event_time": "2025-05-16 13:39:12", "likes": 2958, "comments": 33, "shares": 108}    |
|{"platform": "Twitter", "user_id": "user_tw_127", "tweet_id": "tweet_95785", "event_time": "2025-05-16 13:39:14", "likes": 1646, "retweets": 244, "replies": 217}|
|{"platform": "TikTok", "user_id": "user_tt_489", "video_id": "vid_11461", "event_

-------------------------------------------
Batch: 36
-------------------------------------------
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"platform": "Facebook", "user_id": "user_fb_481", "post_id": "post_3675", "event_time": "2025-05-16 13:39:15", "likes": 2334, "comments": 518, "shares": 140}|
|{"platform": "Instagram", "user_id": "user_ig_279", "post_id": "post_7995", "event_time": "2025-05-16 13:39:16", "likes": 1784, "comments": 376, "shares": 73}|
|{"platform": "Instagram", "user_id": "user_ig_240", "post_id": "post_6108", "event_time": "2025-

-------------------------------------------
Batch: 37
-------------------------------------------
+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                   |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"platform": "TikTok", "user_id": "user_tt_152", "video_id": "vid_37669", "event_time": "2025-05-16 13:39:18", "likes": 487, "shares": 260, "comments": 577}|
+------------------------------------------------------------------------------------------------------------------------------------------------------------+



25/05/16 13:42:28 WARN StandaloneAppClient$ClientEndpoint: Connection to f04d2745dc57:7077 failed; waiting for master to reconnect...
25/05/16 13:42:28 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
25/05/16 13:42:28 WARN StandaloneAppClient$ClientEndpoint: Connection to f04d2745dc57:7077 failed; waiting for master to reconnect...


In [6]:
query.stop()
sc.stop()